In [ ]:
# Mount drive folder in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
from sklearn.impute import KNNImputer

lmap = lambda func, *iterable: list(map(func, *iterable))

# Path to data folder
path_to_data = 'data'

imputation_way = 'KNNImputer' # Mean or KNNImputer

In [ ]:
raw_files = os.listdir(path=path_to_data+'/raw')
raw_files

#Raw data



In [ ]:
def read_csv(path):
  return pd.read_csv(path_to_data+path, sep = ';', index_col=0, low_memory=False, parse_dates=True, skiprows=9)

raw_data = [read_csv('/raw/'+file) for file in raw_files]

In [ ]:
raw_data[0]

#Remove Unnamed column

In [ ]:
def remove_unnamed(data):
  return data.loc[:, ~data.columns.str.contains('^Unnamed')]

correct_data = [remove_unnamed(data) for data in raw_data]

In [ ]:
correct_data[0]

#Select desired columns

In [ ]:
select_columns = [
                  'Hora Medicao',
                  'PRECIPITACAO TOTAL, HORARIO(mm)', 
                  'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA(mB)', 
                  'RADIACAO GLOBAL(Kj/m²)', 
                  'TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)', 
                  'TEMPERATURA MAXIMA NA HORA ANT. (AUT)(°C)',
                  'TEMPERATURA MINIMA NA HORA ANT. (AUT)(°C)',
                  'TEMPERATURA DO PONTO DE ORVALHO(°C)',
                  'UMIDADE RELATIVA DO AR, HORARIA(%)',
                  'UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)',
                  'UMIDADE REL. MIN. NA HORA ANT. (AUT)(%)',
                  'VENTO, DIRECAO HORARIA (gr)(° (gr))',
                  'VENTO, RAJADA MAXIMA(m/s)',
                  'VENTO, VELOCIDADE HORARIA(m/s)'
                  ]

data_selected = [data.loc[:, select_columns] for data in correct_data]

In [ ]:
data_selected[0].head()

#Rename columns

In [ ]:
columns_rename = {
  'Hora Medicao': 'Hora da medição',
  'PRECIPITACAO TOTAL, HORARIO(mm)': 'Precipitação', 
  'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA(mB)': 'Pressão atmosférica (nível da estação)', 
  'RADIACAO GLOBAL(Kj/m²)': 'Radiação', 
  'TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)': 'Temperatura', 
  'TEMPERATURA MAXIMA NA HORA ANT. (AUT)(°C)': 'Temperatura máxima',
  'TEMPERATURA MINIMA NA HORA ANT. (AUT)(°C)': 'Temperatura mínima',
  'TEMPERATURA DO PONTO DE ORVALHO(°C)': 'Temperatura ponto de orvalho',
  'UMIDADE RELATIVA DO AR, HORARIA(%)': 'Umidade relativa',
  'UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)': 'Umidade relativa máxima',
  'UMIDADE REL. MIN. NA HORA ANT. (AUT)(%)': 'Umidade relativa mínima',
  'VENTO, DIRECAO HORARIA (gr)(° (gr))': 'Direção do vento',
  'VENTO, RAJADA MAXIMA(m/s)': 'Vento rajada máxima',
  'VENTO, VELOCIDADE HORARIA(m/s)': 'Velocidade do vento',
}

data_renamed = [data.rename(columns=columns_rename) for data in data_selected]

In [ ]:
data_renamed[0].head()

#Data verification

In [ ]:
data_to_verify = data_renamed[0]

In [ ]:
# Amount of missing data
data_to_verify.isna().sum()

In [ ]:
# Percentage of missing data
data_to_verify.isna().sum()/len(data_to_verify)*100

In [ ]:
# Percentage of missing data
for index, data in enumerate(data_renamed):
  print(raw_files[index])
  print(data.isna().sum()/len(data)*100,end = "\n\n")

## Data imputation

using simple average or KNN imputer

In [ ]:
data_renamed[0].isna().sum()

In [ ]:
# Mean way
def InputWithMean(data):
  return data.fillna(data.mean())

# KNN Imputer way
def InputWithKNN(data):
  index = data.index
  columns = data.columns
  imputer = KNNImputer()
  imputer.fit(data)
  Xtrans = imputer.transform(data)
  return pd.DataFrame(Xtrans, columns=columns, index=index)

imputation_ways = {
    'Mean': InputWithMean,
    'KNNImputer':InputWithKNN
}

imputation = imputation_ways[imputation_way]

data_full_filled = [imputation(data) for data in data_renamed]

In [ ]:
data_full_filled[0].isna().sum()

# Join cities' data

In [ ]:
def getCityName(fileName):
  return fileName.split('.')[0]

cities_names = list(map(getCityName, raw_files))

In [ ]:
def addSufix(sufix, data):
  return data.add_suffix(' ' + sufix)

data_with_cities = [addSufix(cities_names[index], data) for index, data in enumerate(data_full_filled)]
data_concatenated = pd.concat(data_with_cities, axis=1)
data_concatenated.head()

# Save preprocessed data

In [ ]:
# Save separeted files
for index, data in enumerate(data_full_filled):
  file_name = raw_files[index]
  data.to_csv(path_to_data+'/preprocessed/'+file_name, sep=';')

# Save concatenated file
data_concatenated.to_csv(path_to_data+'/preprocessed/alldata.csv', sep=';')